In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp
import sklearn as sk
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import StratifiedKFold


In [5]:
df = pd.read_csv('train.csv')


# Data exploration
#pp.ProfileReport(df)
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [101]:
#concat train and test data
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')
df = pd.concat([df_train, df_test], axis=0, ignore_index=True)
df.to_csv('train_concat.csv', index=False)


## Preprocessing function

In [34]:
df = pd.read_csv('test.csv')
df.head()   

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [38]:
df = pd.read_csv('train.csv')
def preprocessing(df):

    def calculate_group_size(passenger_id, df):
        # Extract the prefix from the 'PassengerId'
        prefix = passenger_id.split('_')[0]
        
        # Count the occurrences of the same prefix in the entire DataFrame
        group_size = len(df[df['PassengerId'].str.startswith(prefix)])
        
        return group_size

    # Apply the function to create the 'TravelGroupSize' column
    df['TravelGroupSize'] = df.apply(lambda row: calculate_group_size(row['PassengerId'], df), axis=1)

    # Filling missing values for HomePlanet
    df['HomePlanet'] = df['HomePlanet'].fillna('Missing')

    # Filling missing values for CryoSleep
    df['CryoSleep'] = df['CryoSleep'].fillna('Missing')

    # Filling missing values for Cabin
    df['Cabin'] = df['Cabin'].fillna('Missing')
    
    # Cabin split
    mask = df['Cabin'] != "Missing"

    # Split 'Cabin' into 'Deck', 'Num', and 'Side' columns
    split_values = df['Cabin'].str.split('/', expand=True)

    # Assign the split values to 'Deck' and 'Side' for non-"Unknown" rows
    df.loc[mask, 'Deck'] = split_values[0]
    df.loc[mask, 'Num'] = split_values[1]
    df.loc[mask, 'Side'] = split_values[2]

    # Fill missing
    df['Deck'] = df['Deck'].fillna("Missing")
    df['Num'] = df['Num'].fillna("Missing")
    df['Side'] = df['Side'].fillna("Missing")

    # Filling missing values for Destination
    df['Destination'] = df['Destination'].fillna('Missing')

    # Filling missing values for Age
    df['Age'] = df['Age'].fillna(df['Age'].mean())

    # Filling missing values for VIP
    df['VIP'] = df['VIP'].fillna(False)

    # Filling missing values for amenities
    df[['FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'RoomService']] = df[['FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'RoomService']].fillna(0)
    
    # Filling missing values with NaN
    cols_with_missing = df.columns[df.isin(['Missing']).any()].tolist()
    for col in cols_with_missing:
        df[col] = df[col].replace('Missing', np.nan)



    # Less important features
    df = df.drop(columns=['Name', 'PassengerId', 'Cabin'])
    df = df.astype({
        'CryoSleep': 'bool',
        'VIP': 'bool',
        'FoodCourt': 'int16',
        'ShoppingMall': 'int16',
        'Spa': 'int16',
        'VRDeck': 'int16',
        'RoomService': 'int16',
        'Age': 'int8',
        'TravelGroupSize': 'int8',
        #'Transported': 'bool'
    })

    return df

# Assuming you have a DataFrame named 'df' that you want to preprocess
#change path if needed for concat file
df = preprocessing(df)
df.to_csv('train_preprocessed.csv', index=False)

In [42]:
df = pd.read_csv('train_preprocessed.csv')
# Separate features and target variable
X = df.drop(columns=["Transported"])
y = df["Transported"]

# Define numeric and categorical features
numeric_features = ['Age', 'TravelGroupSize','FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'RoomService', 'Num']
categorical_features = ['HomePlanet', 'Deck', 'Side', 'Destination', 'VIP', 'CryoSleep']

# Create transformers for numeric and categorical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Use ColumnTransformer to apply transformers to the appropriate columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define a list of classifiers and their respective hyperparameter grids
classifiers = {
     
    #with non concat with moneatary columns
    # Best SVM Model: {'classifier__C': 15, 'classifier__class_weight': 'balanced', 'classifier__gamma': 0.005, 'classifier__kernel': 'rbf', 'classifier__probability': True, 'classifier__shrinking': True}
    # Best SVM Accuracy: 0.80007
    # 'SVM': (SVC(), {
    #      'classifier__C': [15],
    #      'classifier__kernel': ['rbf'],
    #      'classifier__gamma': [0.005],
    #      'classifier__probability': [True],
    #      'classifier__class_weight': ['balanced'],
    #     'classifier__shrinking': [True],
    # }),


    
    #.79 with non concat with moneatary columns
    # 'Logistic Regression': (LogisticRegression(),
    #                             {'classifier__C': [10],
    #                             'classifier__solver': ['saga'],
    #                             'classifier__max_iter': [10000],
    #                             'classifier__class_weight': [None],
    #                             'classifier__tol': [1e-5]}),

   

    # Best Decision Tree Model: {'classifier__class_weight': None, 'classifier__criterion': 'entropy', 'classifier__max_depth': 14, 'classifier__max_features': None, 'classifier__min_samples_leaf': 8, 'classifier__min_samples_split': 5, 'classifier__splitter': 'random'}
    # Best Decision Tree Accuracy: 0.78039
    # 'Decision Tree': (DecisionTreeClassifier(),
    #                    {'classifier__max_depth': [14],
    #                     'classifier__min_samples_split': [5],
    #                     'classifier__min_samples_leaf': [8],
    #                     'classifier__max_features': [None],
    #                     'classifier__criterion': ['entropy'],
    #                     'classifier__splitter': ['random'],
    #                     'classifier__class_weight': [None]}),
    
    
    
    
    # with non concat with moneatary columns
    # Best Random Forest Model: {'classifier__bootstrap': True, 'classifier__class_weight': 'balanced', 'classifier__criterion': 'gini', 'classifier__max_depth': 15, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 5, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 200, 'classifier__n_jobs': -1, 'classifier__random_state': 42}
    # Best Random Forest Accuracy: 0.80398

    # 'Random Forest': (RandomForestClassifier(), {
    # 'classifier__n_estimators': [150, 200, 250],
    # 'classifier__max_depth': [5,10,15],
    # 'classifier__max_features': ['auto', 'sqrt', 'log2'],
    # 'classifier__min_samples_split': [2, 5, 10],
    # 'classifier__min_samples_leaf': [1, 5, 10],
    # 'classifier__bootstrap': [True, False],
    # 'classifier__criterion': ['gini', 'entropy'],
    # 'classifier__class_weight': [None, 'balanced'],
    # 'classifier__random_state': [42],
    # 'classifier__n_jobs': [-1]
    #     }),
    
    # with non concat with moneatary columns
    # Best Gradient Boosting Model: {'classifier__learning_rate': 0.18999999999999995, 'classifier__loss': 'deviance', 'classifier__max_depth': 3, 'classifier__max_features': 'log2', 'classifier__min_samples_leaf': 8, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 450, 'classifier__random_state': 42, 'classifier__subsample': 1.0, 'classifier__warm_start': True}
    # Best Gradient Boosting Accuracy: 0.80754

#     'Gradient Boosting': (GradientBoostingClassifier(), 
#                             {
#                             'classifier__learning_rate': [0.19],
#                             'classifier__loss': ['deviance'],
#                             'classifier__max_depth': [3],
#                             'classifier__max_features': ['log2'],
#                             'classifier__min_samples_leaf': [8],
#                             'classifier__min_samples_split': [2],
#                             'classifier__n_estimators': [450],
#                             'classifier__random_state': [42],
#                             'classifier__subsample': [1.0],
#                             'classifier__warm_start': [True]}
# ),



    # 'Gradient Boosting': (GradientBoostingClassifier(), 
    #                        {'classifier__n_estimators': list(range(440, 500, 20)),
    #                         'classifier__learning_rate': list(np.arange(0.15, 0.22, 0.01)),
    #                         'classifier__max_depth': list(range(2, 6)),
    #                         'classifier__min_samples_split': list(range(5, 10)),
    #                         'classifier__min_samples_leaf': list(range(5, 10)),
    #                         'classifier__subsample': [1.0],
    #                         'classifier__max_features': ['log2'],
    #                         'classifier__loss': ['deviance'],
    #                         'classifier__warm_start': [True],
    #                         'classifier__random_state': [42],
    #                         }),
    # Add other classifiers here
}


# Create a dictionary to store the best models
best_models = {}
classifier_accuracies = {}

# Iterate through the classifiers and perform GridSearchCV
for clf_name, (clf, param_grid) in classifiers.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', clf)])
    
    stratified_kfold = StratifiedKFold(n_splits=11, shuffle=True,random_state=42)

    grid_search = GridSearchCV(pipeline, param_grid, cv=stratified_kfold, n_jobs=-1,scoring='accuracy')
    grid_search.fit(X, y)
    
    best_models[clf_name] = grid_search.best_estimator_
    best_accuracy = grid_search.best_score_
    classifier_accuracies[clf_name] = best_accuracy
    print(f'Best {clf_name} Model: {grid_search.best_params_}')
    print(f'Best {clf_name} Accuracy: {grid_search.best_score_:.5f}')
  


Best Gradient Boosting Model: {'classifier__learning_rate': 0.19, 'classifier__loss': 'deviance', 'classifier__max_depth': 3, 'classifier__max_features': 'log2', 'classifier__min_samples_leaf': 8, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 450, 'classifier__random_state': 42, 'classifier__subsample': 1.0, 'classifier__warm_start': True}
Best Gradient Boosting Accuracy: 0.81077


In [ ]:
final_model=best_models['Gradient Boosting']
print(final_model)

In [43]:
final_model=best_models['Gradient Boosting']
final_model.fit(X,y)


X_test = pd.read_csv('test_preprocessed.csv')


# Load the original test data
original_test_data = pd.read_csv('test.csv')

# Extract the "PassengerId" column
passenger_id_df = original_test_data[['PassengerId']]


y_pred = final_model.predict(X_test)

# Create a DataFrame for the predicted "Transported" values
predicted_df = pd.DataFrame({'Transported': y_pred})

# Concatenate the "PassengerId" and predicted "Transported" DataFrames
submission_df = pd.concat([passenger_id_df, predicted_df], axis=1)

# Save the concatenated DataFrame to a CSV file
submission_df.to_csv('submission_grad_boost.csv', index=False)

In [ ]:
df=pd.read_csv('submission_grad_boost.csv')
df.head(50)